In [1]:
# libraries used
import numpy as np
import pandas as pd
from tvDatafeed import TvDatafeed, Interval
from matplotlib import pyplot as plt
import csv

### INDICATORS FUNCTIONS

In [2]:
# Indicators functions

# SMA :- Simple Moving Average.
def SMA(col: pd.Series, period: int = 50):
    """
    Calculate SMA using given column
    col : pd.Series
    period : int (default)
    """
    return col.rolling(period).mean().round(2)

# RSI :- Relative Strength Index.
def RSI(change : pd.Series, period: int = 14):
    """
    calculate RSI using price change data.
    change : pd.Sereies
    period : int
    """
    gain = change.where(change > 0, 0)
    loss = change.where(change < 0 , 0)
    loss = loss.abs()
    avgain = gain.rolling(period).mean()
    avloss = loss.rolling(period).mean()
    rs = avgain / avloss
    rsi = 100 - (100 / (1 + rs))
    return rsi.round(2)

# ADX :- Average Directional Index
def ADX(high, low, close, period:int = 14):
    TR = pd.DataFrame({
        'a': high - low,
        'b': abs(high - close.shift()),
        'c': abs(low - close.shift())
    }).max(axis=1)

    # Calculate the Directional Movement (DM)
    DMplus = (high - high.shift()) > (low.shift() - low)
    DMminus = (low.shift() - low) > (high - high.shift())

    # Calculate the True Directional Indicator (+DI) and (-DI)
    TR14 = TR.rolling(window=period).sum()
    DM14plus = DMplus.rolling(window=period).sum()
    DM14minus = DMminus.rolling(window=period).sum()

    DIplus14 = DM14plus / TR14 * 100
    DIminus14 = DM14minus / TR14 * 100

    # Calculate the Directional Movement Index (DMI)
    DX14 = abs(DIplus14 - DIminus14) / (DIplus14 + DIminus14) * 100
    ADX14 = DX14.rolling(window=period).mean()
    return ADX14.round(2)


# calcualte volume weigthed average price.
def VWAP(price: pd.Series, vol:pd.Series, period:int = 14) -> pd.Series:
    num = price * vol
    vmp = num.rolling(period).sum() 
    sv = vol.rolling(period).sum()
    return vmp/sv

# calcualting macd.
def MACD():
    pass

#### INITIALIZING TV-DATA-FEED

In [3]:
tv = TvDatafeed('mohitrathor7729', 'Mohit7729@#')
tv

## NIFTY 100 SCANS
### CRITERIA : 
 + #### 5 & 50 moving average should be minimum.

In [4]:
# WORK ON NIFTY 100.
n100 = pd.read_csv('niftymid100.csv', names = ['Symbol', 'Open', 'High', 'Low', 'Pre Close', 'LTP', 'Change', '%chng' , 'Vol', 'Value', '52W H' , '52W L' , '365D %c', '30D %c'])
n100sym = n100['Symbol']
n100sym

0             SYMBOL \n
1      NIFTY MIDCAP 100
2              SONACOMS
3               MPHASIS
4             MAXHEALTH
             ...       
97                 SAIL
98                  OIL
99            POLICYBZR
100                BHEL
101            PRESTIGE
Name: Symbol, Length: 102, dtype: object

In [ ]:
# short term trading scan
with open('NIFTY100-scan.csv', 'w') as fh:
    j = 1
    writer = csv.writer(fh)
    for i in nft100['SYMBOL']:
        try:
            data = tv.get_hist(symbol= i, exchange= 'NSE', interval= Interval.in_daily, n_bars = 200)
            data['MA5'] = SMA(data['close'], period= 5)
            data['MA50'] = SMA(data['close'])
            data['ADX']  = ADX(data['high'], data['low'], data['close'])
            writer.writerow(list(data.iloc[-1]))
            print(j)
            j += 1
        except Exception as e: print(e, i, 'Data unavailable !')
            
print('scan Completed !!!')